In [1]:
import os
archivos_entrenamiento = os.listdir("Entrenamiento/")
archivo_test = os.listdir("Test/")
#set([os.path.splitext(archivo)[1] for archivo in archivos_entrenamiento]) # obtengo las extensiones de los archivos

In [2]:
class ArchivoTxt:
    def __init__(self, nombre, extension, texto):
        self.nombre = nombre
        self.extension = extension
        self.texto = texto

In [3]:
from tika import parser
from nltk import word_tokenize

def convertir_documento_a_txt(archivo):
    raw = parser.from_file(archivo)
    return raw['content']

#word_tokenize(convertir_pdf_txt("dataset-nlp-plagio-utn/Domótica_Final.pptx"))

In [4]:
def convertir_archivo_a_txt(archivo):
    archivo_nombre, archivo_extension = os.path.splitext(archivo)
    if(archivo_extension != ".pptx"):
        archivo_txt = convertir_documento_a_txt(archivo)
        return ArchivoTxt(archivo_nombre, archivo_extension, archivo_txt)
    
#convertir_archivo_a_txt("dataset-nlp-plagio-utn/MKT 2016 - Alan Szpigiel - TP4.pdf").nombre

In [5]:
from nltk import sent_tokenize # divide en oraciones

archivos_entrenamiento_txt = [convertir_archivo_a_txt("Entrenamiento/" + archivo) for archivo in archivos_entrenamiento]

#for archivo in archivos_entrenamiento_txt:
#    if(archivo != None):
#        archivo.texto = sent_tokenize(archivo.texto.strip())

2020-05-25 12:10:03,867 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [6]:
archivo_test_txt = [convertir_archivo_a_txt("Test/" + archivo) for archivo in archivo_test][0]

In [7]:
# Nombre del archivo de texto procesado
archivo_test_txt.nombre # la extension hace falta?

'Test/TP 3 The experience economy'

In [ ]:
# Nombre y apellido del alumno
import spacy
import itertools
from nltk import line_tokenize

nombre_alumno=''

nlp=spacy.load('es_core_news_sm') # modelo para detectar Entidades (nombres)
texto_dividido_en_lineas = line_tokenize(archivo_test_txt.texto.strip())
lineas_pasadas_por_el_modelo = [[(a.text, a.label_) for a in nlp(linea).ents] for linea in texto_dividido_en_lineas]
lista_flatenizada = list(itertools.chain.from_iterable(lineas_pasadas_por_el_modelo)) #flatten
print(lista_flatenizada)
entidades_reconocidas_como_personas = [texto for (texto,categoria) in lista_flatenizada if categoria == 'PER']

for entidad in entidades_reconocidas_como_personas:
    if entidad in archivo_test_txt.nombre:
        nombre_alumno = entidad

if(nombre_alumno != ''):
    print('Nombre y apellido del alumno: ' + nombre_alumno)
else:
    print(entidades_reconocidas_como_personas)
    #print([nltk.Text(word_tokenize(archivo_test_txt.texto)).concordance(entidad) for entidad in entidades_reconocidas_como_personas])